In [ ]:
import os
import shutil

In [ ]:
try:
    os.mkdir('zip')
except:
    pass
os.chdir('zip')
try:
    os.mkdir('../pdf_new')
except:
    pass
for folder in os.listdir(os.getcwd()):
    if folder.endswith('.pdf'):
        shutil.move(folder,os.getcwd()[:-3]+'pdf_new/'+folder)
    elif folder.endswith('ipynb'):
        continue
    else:            
        for file in os.listdir(os.getcwd()+'/'+folder):
            shutil.move(os.getcwd()+'/'+folder+'/'+file,os.getcwd()[:-3]+'pdf_new/'+file)
        os.rmdir(folder)
os.chdir('../') 
os.rmdir('zip')

In [ ]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime
from csv import writer

In [ ]:
#create folder to save csv files
month=str(datetime.now().month)
year=str(datetime.now().year)
try:
    os.mkdir('csv_'+month+'_'+year)
except:
    pass
#open main page of EHPA
manufacturers_info = BeautifulSoup(requests.get('https://www.heatpumpkeymark.com/?type=109126').content, 'html.parser')
#look for all manufacturers
for manufacturer_info in manufacturers_info.find_all('td'):                                                               
    manufacturer=manufacturer_info.text
    models_info= BeautifulSoup(requests.get('https://www.heatpumpkeymark.com/'+manufacturer_info.a.get('href')).content, 'html.parser')
    #look for models:
    for model in models_info.find_all('td'):
        if model.text!=manufacturer:
            try:
                model_info = BeautifulSoup(requests.get('https://www.heatpumpkeymark.com/'+model.a.get('href')).content, 'html.parser')
            except:
                continue
            for info_col in model_info.find_all(class_='info-coll'):
                if (info_col.find(class_='info-label').span.text)==('Refrigerant'):
                    ref = (info_col.find(class_='info-data').text.replace(' ','').replace('\n',''))
                if (info_col.find(class_='info-label').span.text)==('Mass of Refrigerant'):
                    mass_of_ref = (info_col.find(class_='info-data').text.replace('\n',''))
                if (info_col.find(class_='info-label').span.text)==('Certification Date'):
                    date = (info_col.find(class_='info-data').text.replace(' ','').replace('\n',''))
            #choose correct export method (csv preffered) and get basic data TODO
            for export_method in model_info.find_all('a'):
                if export_method.text.startswith('Export'):
                    soup3 = BeautifulSoup(requests.get('https://www.heatpumpkeymark.com/'+export_method.get('href')).content,'html.parser')
                    #open Download link
                    for link in soup3.find_all('a'):
                        if link.text=='Download':
                            #write to csv
                            foldername='csv_'+month+'_'+year+'/'
                            filename=manufacturer+model.text.replace('/','_')
                            filename=manufacturer+model.text.replace(' ','_')
                            csv_content = requests.get('https://www.heatpumpkeymark.com/'+link.get('href')).content
                            with open(foldername+filename+'.csv', 'wb') as file:
                                file.write(csv_content)
                            with open(foldername+filename+'.csv', 'a') as f:
                                f.write('"","Refrigerant","'+str(ref)+'","0","0","0","0"\r\n')
                                f.write('"","Mass of Refrigerant","'+str(mass_of_ref)+'","0","0","0","0"\r\n')
                                f.write('"","Date","'+str(date)+'","0","0","0","0"\r\n')
                                f.write('"","Manufacturer","'+manufacturer+'","0","0","0","0"\r\n')
                                f.write('"","Modelname","'+model.text+'","0","0","0","0"\r\n')

In [ ]:
import pandas as pd

In [ ]:
df_all=pd.DataFrame()
manufacturers, models, titels, dates, types, refrigerants, mass_of_refrigerants, spl_indoors, spl_outdoors, eta, p_rated, scop, t_biv, tol, p_th_minus7, cop_minus7, p_th_2, cop_2, p_th_7, cop_7, p_th_12, cop_12, p_th_tbiv, cop_tbiv, p_th_tol, cop_tol, rated_airflows, wtols, poffs, ptos, psbs, pcks, supp_energy_types, p_sups, p_design_cools, seers, pdcs_35, eer_35, pdcs_30, eer_30, pdcs_25, eer_25, pdcs_20, eer_20, temperatures= ([] for i in range(45))
lists=[manufacturers, models, titels, dates, types, refrigerants, mass_of_refrigerants, spl_indoors, spl_outdoors, eta, p_rated, scop, t_biv, tol, p_th_minus7, cop_minus7, p_th_2, cop_2, p_th_7, cop_7, p_th_12, cop_12, p_th_tbiv, cop_tbiv, p_th_tol, cop_tol, rated_airflows, wtols, poffs, ptos, psbs, pcks, supp_energy_types, p_sups, p_design_cools, seers, pdcs_35, eer_35, pdcs_30, eer_30, pdcs_25, eer_25, pdcs_20, eer_20, temperatures]
values=['Manufacturer','Modelname','title','Date','application','Refrigerant','Mass of Refrigerant','EN12102_1_001','EN12102_1_002','EN14825_001','EN14825_002','EN14825_003','EN14825_004','EN14825_005','EN14825_008','EN14825_009','EN14825_010','EN14825_011','EN14825_012','EN14825_013','EN14825_014','EN14825_015','EN14825_016','EN14825_017','EN14825_018','EN14825_019','EN14825_020','EN14825_022','EN14825_023','EN14825_024','EN14825_025','EN14825_026','EN14825_027','EN14825_028','EN14825_030','EN14825_031','EN14825_032','EN14825_033','EN14825_034','EN14825_035','EN14825_036','EN14825_037','EN14825_038','EN14825_039']
general_info=['Manufacturer','Modelname','Date','Refrigerant','Mass of Refrigerant']
with os.scandir('csv_11_2022') as dir1:
    for file in dir1:
        j=0 #j: start index; i: end index
        df=pd.read_csv(file)
        try:
            df.loc[df['varName']=='title']
        except:
            print(file,len(df))
            continue
        for model in range(1,len(df.loc[df['varName']=='title'])+1):
            try:
                i=(df.loc[df['varName']=='title'].index[model])
            except:
                i=len(df)
            #df1 is dataframe of each model
            df1=(df.iloc[j:i])
            j=i
            for temperature in ['low','high']:
                #df2 is dataframe of each climate and temperature
                df2=df1.loc[(df1['climate']>=3) & (df1['climate']<=4)]
                if temperature=='low':
                    df2.loc[df2['temperature']==4,'temperature']=6
                    df2=df2.loc[(df2['temperature']==6)]
                    temperatures.append('low')
                else:
                    df2.loc[(df2['temperature']==5),'temperature']=7
                    df2=df2.loc[(df2['temperature']==7)]
                    temperatures.append('high')
                for value, lst in zip(values,lists):
                    if value in general_info:
                        read_value=df.loc[df['varName']==value,'value'].values[0]
                    else:
                        try:
                            read_value=df2.loc[df2['varName']==value,'value'].values[0]
                        except:
                            read_value=''
                    lst.append(read_value)
df_all['manufacturers']=manufacturers
df_all['models']=models
df_all['titels']=titels
df_all['dates']=dates
df_all['types']=types
df_all['refrigerants']=refrigerants
df_all['mass_of_refrigerants']=mass_of_refrigerants
df_all['spl_indoor']=spl_indoors
df_all['spl_outdoor']=spl_outdoors
df_all['eta']=eta
df_all['p_rated']=p_rated
df_all['scop']=scop
df_all['t_biv']=t_biv
df_all['tol']=tol
df_all['p_th_minus7']=p_th_minus7
df_all['cop_minus7']=cop_minus7
df_all['p_th_2']=p_th_2
df_all['cop_2']=cop_2
df_all['p_th_7']=p_th_7
df_all['cop_7']=cop_7
df_all['p_th_12']=p_th_12
df_all['cop_12']=cop_12
df_all['p_th_tbiv']=p_th_tbiv
df_all['cop_tbiv']=cop_tbiv
df_all['p_th_tol']=p_th_tol
df_all['cop_tol']=cop_tol
df_all['rated_airflows']=rated_airflows
df_all['wtols']=wtols
df_all['poffs']=poffs
df_all['ptos']=ptos
df_all['psbs']=psbs
df_all['pcks']=pcks
df_all['supp_energy_types']=supp_energy_types
df_all['p_sups']=p_sups
df_all['p_design_cools']=p_design_cools
df_all['seers']=seers
df_all['pdcs_35']=pdcs_35
df_all['eer_35']=eer_35
df_all['pdcs_30']=pdcs_30
df_all['eer_30']=eer_30
df_all['pdcs_25']=pdcs_25
df_all['eer_25']=eer_25
df_all['pdcs_20']=pdcs_20
df_all['eer_20']=eer_20
df_all['temperatures']=temperatures


In [ ]:
df=pd.read_csv('../output/database.csv')
df=df.loc[df['eta'].isna()==0]
df=df.drop_duplicates(subset=['types', 'eta', 'p_rated',
    'scop', 't_biv', 'tol', 'p_th_minus7', 'cop_minus7', 'p_th_2', 'cop_2',
    'p_th_7', 'cop_7', 'p_th_12', 'cop_12', 'p_th_tbiv', 'cop_tbiv',
    'p_th_tol', 'cop_tol', 'wtols', 'poffs', 'ptos',
    'psbs', 'pcks', 'supp_energy_types', 'p_sups', 'p_design_cools',
    'seers', 'pdcs_35', 'eer_35', 'pdcs_30', 'eer_30', 'pdcs_25', 'eer_25',
    'pdcs_20', 'eer_20', 'temperatures'])
df.sort_values(by=['manufacturers','models'], inplace=True,key=lambda col: col.str.lower())
df=df.loc[df['temperatures']=='low'].merge(df.loc[df['temperatures']=='high'],on='titels')
df.to_csv('../output/database_reduced.csv',index=False)